## 10. Bash Scripting Finale: Auto Web Setup

We are putting into practice what we've learned about bash scripting and what we do here is going to be the base of many other automation tools.

### Script Setup

* Bring up the VMs: scriptbox, web01, web02 and web03

* Log in to the *scriptbox* VM and the root user

* Go to `/opt/scripts/` and create a directory: `mkdir remote_websetup`.

* In the new directory, create a host file: `vim remhosts` and add the IP addresses or hostnames of the VMs you are connecting (**targets**):

    ```
      1 web01
      2 web02
      3 web03
    ```

* Use a `for` loop to test the login to these machines. Use the `;` to separate lines in the same command line:

    ```
      # for printing:
      for host in `cat remhosts`; do echo $host; done

      # for test the login:
      for host in `cat remhosts`; do ssh devops@$host hostname; done
      for host in `cat remhosts`; do ssh devops@$host uptime; done
    ```

* Copy/Create the script for websetup using variables (`../3_vars_websetup.sh`) to the `remote_websetup` directory. The script contains the following:

    ```
        1 #!/bin/bash
        2
        3 # Variables
        4 PACKAGE="httpd wget unzip"
        5 SVC="httpd"
        6 URL='https://www.tooplate.com/zip-templates/2098_health.zip'
        7 ART_NAME='2098_health'
        8 TEMPDIR="/tmp/webfiles"
        9
      10 # Installing dependencies (> to send output to null - do not show)
      11 echo "#####################################"
      12 echo "Installing packages"
      13 echo "#####################################"
      14 sudo yum install $PACKAGE -y > /dev/null
      15 echo
      16
      17 # Start and enable services
      18 echo "#####################################"
      19 echo "Start & enable HTTPD Service"
      20 echo "#####################################"
      21 sudo systemctl start $SVC
      22 sudo systemctl enable $SVC
      23 echo
      24
      25 # Creating a temporary directory
      26 echo "#####################################"
      27 echo "Starting artifact deployment"
      28 echo "#####################################"
      29 mkdir -p $TEMPDIR
      30 cd $TEMPDIR
      31 echo
      32
      33 # Get and copy webpage template to the /var/www/html directory
      34 wget $URL > /dev/null
      35 unzip $ART_NAME.zip > /dev/null
      36 sudo cp -r $ART_NAME/* /var/www/html/
      37 echo
      38
      39 # Bounce service
      40 echo "#####################################"
      41 echo "Restarting HTTPD Service"
      42 echo "#####################################"
      43 systemctl restart $SVC
      44 echo
      45
      46 # Clean up temporary files
      47 echo "#####################################"
      48 echo "Removing temporary files"
      49 echo "#####################################"
      50 rm -rf $TEMPDIR
      51
      52 # Print status of the service
      53 sudo systemctl status $SVC
      54
      55 # Show the content of the /var/www/html directory
      56 ls /var/www/html/
    ```

* Rename the file: `mv 3_vars_websetup.sh multios_websetup.sh`.

* Modify the file to make it multiOS (currently is only for CentOS): for CentOS and Ubuntu:

    - We need to add some verifications using `if` conditions

    - Comment (`#`) the variable declaration for `PACKAGE` and `SVC` (lines 4 and 5)

    - Before line 10 of the script, we add an `if` condition to verify whether `yum` is part of the VM (CentOS system) or not (Ubuntu system)
    
    - The code will run following the conditional, i.e., all CentOS-related packages and software are going to be installed if the condition is True. If False, then all packages and software related to Ubuntu should be included:

    ```
          # Verify if the system is CentOS or Ubuntu
          yum --help &> /dev/null

          if [ $? -eq 0 ]
          then
            
            echo "Running setup on CentOS system"

            # Set variables for CentOS
            PACKAGE="httpd wget unzip"
            SVC="httpd"
            ...

          else

            echo "Running setup on Ubuntu system"

            # Set variables for Ubuntu
            PACKAGE="apache2 wget unzip"
            SVC="apache2"
            
            # Install dependencies (> to send output to null - do not show)
            echo "#####################################"
            echo "Installing packages"
            echo "#####################################"
            sudo apt update
            sudo apt install $PACKAGE -y > /dev/null
            ...
          fi
    ```

### Run the Script Remotely

The command to push files to Linux machines is `scp`, which uses `ssh` (if the key login is working), and use a text file giving the specific path:

`scp filename.txt username@hostname:/path/`

Example:

`scp testfile.txt devops@web01:/tmp/`

So, we need a script to push the `multios_websetup.sh` script and then execute it:

* Create a file in the `remote_websetup` directory: `vim web_deploy.sh`

```
        1 #!/bin/bash
        2
        3 USR='devops'
        4
        5 for host in `cat remhosts`
        6 do
        7    echo "###########################################"
        8    echo "Connecting to $host"
        9    echo "Pushing the script to $host"
        10   scp multios_websetup.sh $USR@$host:/tmp/
        11   echo "Executing the script on $host"
        12   ssh $USR@$host sudo rm -rf /tmp/multios_websetup.sh
        13   echo "###########################################"
        14   echo
        15 done
```

* Make the deployment file executable: `chmod +x web_deploy.sh`

* Run the file: `./web_deploy.sh`